# Weaviate Challenge Solution
## William Pugsley

We will explore the [LinkedIn job postings (2023-2024) dataset](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/) from Kaggle. The purpose of this project is to cluster job postings by machine learning, artificial intelligence, and data science.

Requires the postings.csv file from Kaggle to be in the directory.



Explore data

Engineer data (salaries, no descriptions, etc.)

Add files to Weaviate server

Find jobs closest to data science, machine learning, AI

plot scatter plots of salary vs. views, views vs. applies, 

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import os
import weaviate
import weaviate.classes as wvc

# Preliminaries

In [16]:
#reads a file that holds all the API keys and URLs

# open the file in read mode
with open("secrets.txt", 'r') as file:
    # read lines from the file
    lines = file.readlines()

secrets = {}

# iterate through each line and split key-value pairs
for line in lines:
    key, value = line.strip().split(',')
    secrets[key.strip()] = value.strip()

# Data Exploration

We will start by exploring the dataset, extracting the relevant instances, and engineering new fields.

In [18]:
df_postings_raw = pd.read_csv("postings.csv")

In [19]:
df_postings_raw.head(2)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.715990e+12,NaN,NaN,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.715450e+12,NaN,NaN,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY


In [20]:
df_postings_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [23]:
#where are the min and max salaries both not NaN?
not_na_col1 = df_postings_raw["min_salary"].notna()
not_na_col2 = df_postings_raw["max_salary"].notna()

both_not_na = not_na_col1 & not_na_col2

# Count the number of rows where both columns are not NaN
num_rows = both_not_na.sum()
print(num_rows)

29793


Every instance that has a min salary also has a max salary. What about for the med salary field?

In [24]:
not_na_col1 = df_postings_raw["min_salary"].notna()
not_na_col2 = df_postings_raw["med_salary"].notna()

both_not_na = not_na_col1 & not_na_col2

# Count the number of rows where both columns are not NaN
num_rows = both_not_na.sum()
print(num_rows)

0


There are no instance of a job posting having a median salary and a min/max salary. 

In [62]:
df_postings = df_postings_raw.copy()

In [63]:
df_postings.loc[df_postings_raw["min_salary"].notna(), "med_salary"] = (df_postings_raw["min_salary"][df_postings_raw["min_salary"].notna()]
                                                                         + df_postings_raw["max_salary"][df_postings_raw["max_salary"].notna()])/2
#we can drop the max_salary and min_salary columns since we combined them
df_postings = df_postings.drop(["min_salary", "max_salary"], axis=1)

We are only interested in job postings that have descriptions, titles, salaries, locations, views, applications, and experience levels. We will remove all other columns

In [64]:
useful_fields = ["med_salary", "description", "title", "job_id", "location", "applies", "views", "formatted_experience_level"]

In [65]:
df_postings = df_postings.drop([col for col in df_postings.columns if col not in useful_fields], axis=1)

In [68]:
df_postings = df_postings.dropna(axis=0, how='any').reset_index(drop=True)

In [60]:
df_postings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       6718 non-null   int64  
 1   job_id                      6718 non-null   int64  
 2   title                       6718 non-null   object 
 3   description                 6718 non-null   object 
 4   location                    6718 non-null   object 
 5   views                       6718 non-null   float64
 6   med_salary                  6718 non-null   float64
 7   applies                     6718 non-null   float64
 8   formatted_experience_level  6718 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 472.5+ KB


# Adding Data to Weaviate Database

In [81]:
#connect to weaviate cloud server
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=secrets["WCD_URL"],
    auth_credentials=weaviate.auth.AuthApiKey(secrets["WCD_API_KEY"]),
    #headers={
    #    "X-OpenAI-Api-Key": secrets["OPENAI_API_KEY"]
    #}
)

In [82]:
#create the collection
postings = client.collections.create(
        name="Postings",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
        #generative_config=wvc.config.Configure.Generative.openai()
    )

We will only take a subset of samples for demonstration purposes.

In [83]:
postings_objs = list()
for idx, row in df_postings.sample(100, replace=False).iterrows(): #iterate over datapoints
    datapoint = {}
    for col in df_postings.columns: #add every column to the database
        datapoint[col] = row[col]
        
    postings_objs.append(datapoint)

In [84]:
postings = client.collections.get("Postings")
postings.data.insert_many(postings_objs) #insert the data
client.close()

WeaviateInsertManyAllFailedError: Every object failed during insertion. Here is the set of all errors: API Key: no api key found neither in request header: X-Openai-Api-Key nor in environment variable under OPENAI_APIKEY

# Query the Database

Now that we have the data in the Weaviate database, we can query it for job postings related to machine learning and AI.

In [80]:
#connect to weaviate cloud server
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=secrets["WCD_URL"],
    auth_credentials=weaviate.auth.AuthApiKey(secrets["WCD_API_KEY"]),
    #headers={
    #    "X-OpenAI-Api-Key": secrets["OPENAI_API_KEY"]
    #}
)

try:
    postings = client.collections.get("Postings")

    response = postings.query.near_text(
        query="machine learning",
        limit=2
    )

    print(response.objects[0].properties)  # Inspect the first object

finally:
    client.close()

WeaviateQueryError: Query call with protocol GRPC search failed with message explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: connection to: OpenAI API failed with status: 429 error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..